# Segmentation Ensemble models

> Ensemble with encoder ResNet34 and FPN, UNET backbones.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from steel_segmentation.metadata import *
from steel_segmentation.masks import *
from steel_segmentation.dataloaders import *
from steel_segmentation.models.trainer import *
from steel_segmentation.models.predict import *

from fastai.vision.all import *
from matplotlib import pyplot as plt

import warnings
import random
import os
import cv2
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import segmentation_models_pytorch as smp

seed_everything()

In [ ]:
torch.device("cuda:0").type

'cuda'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Thu Feb 11 08:22:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                 ERR! |
| 26%   24C    P8     6W / 180W |      4MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = torch.device("cuda")

## UNET model

In [ ]:
dls = get_segmentation_dls(4, (256, 1600), with_btfms=False)

In [ ]:
unet_trainer = unet_learner(dls=dls, arch=resnet34, metrics=seg_metrics, pretrained=True)
unet_trainer.model_dir = models_dir
unet_trainer = unet_trainer.load("ResNet34-Unet-256-stage5")

In [ ]:
unet_model = unet_trainer.model
unet_model.to(device)
unet_model = unet_model.eval()

## FPN model

In [ ]:
fpn_model = smp.FPN("resnet34", encoder_weights='imagenet', classes=4, activation=None)

In [ ]:
loaded_dict = torch.load(models_dir/"kaggle-FPN-ResNet34.pth")
fpn_model.load_state_dict(loaded_dict["state_dict"], strict=True)

<All keys matched successfully>

In [ ]:
fpn_model.to(device)
fpn_model = fpn_model.eval()

## Inference

In [ ]:
def post_process(probability, threshold, min_size):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((256, 1600), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [ ]:
sample_submission_path = path / 'sample_submission.csv' # file da cui pescare i ImageId
test_data_folder = path / "test_images" # cartella dove trovare le immagini

In [ ]:
testset = get_test_dls(sample_submission_path, batch_size=2)

In [ ]:
best_threshold = 0.5
min_size = 3000

In [ ]:
min_sizes = [800, 800, 3000, 2500]

In [ ]:
name, x = next(iter(testset))
x.shape

torch.Size([2, 3, 256, 1600])

In [ ]:
fpn_preds = torch.sigmoid(fpn_model(x.to(device)))
fpn_preds.shape

torch.Size([2, 4, 256, 1600])

In [ ]:
unet_preds = torch.sigmoid(unet_model(x.to(device)))
unet_preds.shape

torch.Size([2, 5, 256, 1600])

In [ ]:
unet_preds[:, 1:].shape

torch.Size([2, 4, 256, 1600])

In [ ]:
# start prediction
predictions = []
for i, batch in enumerate(tqdm(testset)):
    fnames, images = batch
    images = images.to(device)
    
    # FPN
    fpn_preds = torch.sigmoid(fpn_model(images))
    fpn_preds = fpn_preds.detach().cpu().numpy()
    
    # UNET
    unet_preds = torch.sigmoid(unet_model(images))
    unet_preds = unet_preds[:, 1:].detach().cpu().numpy()
    
    batch_preds = (fpn_preds + unet_preds) / 2
    for fname, preds in zip(fnames, batch_preds):
        for cls, pred in enumerate(preds):
            min_size = min_sizes[cls]
            pred, num = post_process(pred, best_threshold, min_size)
            rle = mask2rle(pred)
            name = fname + f"_{cls+1}"
            predictions.append([name, rle])

# save predictions to submission.csv
df = pd.DataFrame(predictions, columns=['ImageId_ClassId', 'EncodedPixels'])
df.to_csv(sub_path/"ensemble_submission_thresholds.csv", index=False)

100%|██████████| 2753/2753 [08:45<00:00,  5.24it/s]


In [ ]:
df.head()

,ImageId_ClassId,EncodedPixels
0,0000f269f.jpg_1,
1,0000f269f.jpg_2,
2,0000f269f.jpg_3,146418 5 146424 1 146577 30 146608 1 146610 1 146612 21 146638 48 146732 69 146802 1 146808 1 146810 1 146812 1 146820 125 146962 1 146976 225 147208 249 147461 252 147716 253 147971 254 148227 254 148483 254 148739 254 148995 254 149251 254 149507 254 149763 254 150019 254 150275 254 150532 253 150788 253 151045 252 151303 250 151563 246 151848 11 151862 13 151876 189 152152 169 152430 29 152489 1 152491 1 152493 6 152500 7 152557 12
3,0000f269f.jpg_4,
4,000ccc2ac.jpg_1,


## Refrences

